In [ ]:
# | default_exp api

# pingme API
The pingme API is built on top of the pingme.services layer. The API is intended to be a 1-to-1 mapping of services function as a REST API call. That means to implement into this library you should first define your interface into pingme.services then, once complete it can be added here as a call.

In [ ]:
# |hide
import nbdev
from nbdev.showdoc import *

# Included Libraries

In [ ]:
# |export
import uvicorn  # Server for hosting the API, ref: https://www.uvicorn.org/
from fastapi import FastAPI  # library for creating the API
from fastapi.testclient import TestClient  # test client for notebook to test API calls
from fastapi import HTTPException  # for raising exceptions

from pingme.core import settings
from pingme import core
from pingme.pingme_class import Card
from pingme.services import NotificationService

from fastcore.script import (
    call_parse,  # To create a CLI from a function
)

import json  # for parsing json data

Launch the app to run things, using default params which includes /docs for swagger ui

In [ ]:
# |export
app = FastAPI()

In [ ]:
# | export
@app.post("/webhook/default")
def webhook_card_default():
    """
    Send a default card to the webhook, intention is strictly for testing and showcasing.
    """
    try:
        return NotificationService.send_default_card_to_webhook()
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
client = TestClient(app)
response = client.post("/webhook/default")
print(response, json.dumps(response.json(), indent=2))

In [ ]:
# | export
@app.post("/webhook/simple")
def webhook_card_simple(title: str, text: str):
    """
    Send a simple card to the webhook, should be used for most general use cases of sending a message.

    Args:
    title: Title of the card
    text: Text of the card
    """
    try:
        return NotificationService.send_simple_card_to_webhook(title, text)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
client = TestClient(app)
response = client.post("/webhook/simple?title=TestingTitle&text=TestText")
print(response, json.dumps(response.json(), indent=2))

In [ ]:
# | export
@app.post("/webhook/card/")
def webhook_card(card: Card):
    """
    Send a card to the webhook, card defines a card thats installed into the config.yaml. Advanced usage which may not get used.

    Args:
    card: Card object
    """
    try:
        return NotificationService.send_card_to_webhook(card)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
# Test wont work for runner with default config
client = TestClient(app)
response = client.post(
    "/webhook/card",
    json={
        "name": "default",
        "context": {"title": "titlevalue", "text": "textvalue"},
    },
)
print(response, json.dumps(response.json(), indent=2))

In [ ]:
# |export
@app.get(path="/")
@app.get("/help", tags=["help"])
async def help():
    """
    Information on how to use API to users, when they hit the root URL. Ensure when you add new endpoints to update this function, check /docs for info on endpoints.
    """
    return {"msg": "please check /docs for more information on how to use the API"}

In [ ]:
# Test
client = TestClient(app)
response = client.get("/")
print(response.status_code, json.dumps(response.json(), indent=2))

In [ ]:
# |export
@call_parse
def webservice(
    host: str = "127.0.0.1",  # Host to run the server on
    port: int = 5000,  # Port to run the server on"
    config_file: str = None,  # Path to config file"
):
    """Start the PingMe API server."""
    # Override settings if provided
    if config_file is not None:
        settings.config_file = config_file

    # Run using module path instead of app instance to ensure updated config
    uvicorn.run("pingme.api:app", host=host, port=port, reload=core.DEV_MODE)

In [ ]:
# | hide
from nbdev import nbdev_export

nbdev_export()